In [1]:
import keras
from keras.preprocessing import image as image_utils
from imagenet_utils import decode_predictions
from imagenet_utils import preprocess_input
import numpy as np
import cv2
from StringIO import StringIO
import urllib
from PIL import Image
from vgg16 import VGG16
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
orig = image_utils.load_img("panda.JPG", target_size=(224,224))
orig = np.asarray(orig)

In [3]:
image = image_utils.load_img("panda.JPG", target_size=(224,224))
image = image_utils.img_to_array(image)

In [4]:
image = np.expand_dims(image,axis=0)
image = preprocess_input(image)

In [5]:
# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet")
# classify the image
print("[INFO] classifying image...")
preds = model.predict(image)


[INFO] loading network...
K.image_dim_ordering: tf
[INFO] classifying image...


In [6]:
(inID, label, prob) = decode_predictions(preds)[0][0]

In [7]:
layer_output = model.layers[-1].output

In [8]:
loss = keras.backend.mean(layer_output[:, 388])

In [9]:
grads = keras.backend.gradients(loss, model.input)

In [10]:
iterate = keras.backend.function([model.input],[loss, grads[0]])

In [11]:
grad = iterate([image])

In [12]:
grad = grad[1].reshape(224,224,3)

In [13]:
delta = np.sign(grad)
img_pert = orig 

In [15]:
black = np.zeros_like(image)
# load the VGG16 network
print("[INFO] loading network...")
model = VGG16(weights="imagenet")
# classify the image
print("[INFO] classifying image...")
preds = model.predict(black)


[INFO] loading network...
K.image_dim_ordering: tf
[INFO] classifying image...


In [16]:
decode_predictions(preds)

[[(u'n03788365', u'mosquito_net', 0.016361972),
  (u'n03291819', u'envelope', 0.013312053),
  (u'n15075141', u'toilet_tissue', 0.013128777),
  (u'n01737021', u'water_snake', 0.012501626),
  (u'n04209239', u'shower_curtain', 0.0091519803)]]

In [17]:
layer_output = model.layers[-1].output
layer_output

<tf.Tensor 'Softmax_1:0' shape=(?, 1000) dtype=float32>

In [ ]:
x = tf.placeholder(tf.float32, shape=model.input_shape)
y = tf.placeholder(tf.float32, shape=model.output_shape)

predictions = model(x)

y = tf.to_float(tf.equal(predictions, tf.reduce_max(predictions, 1, keep_dims=True))) #compare with its max
y = y / tf.reduce_sum(y, 1, keep_dims=True) #normalise
loss = tf.reduce_mean(categorical_crossentropy(y, predictions))